**Импортируем данные из json файла**

In [53]:
import random as rd
import pandas as pd
import re
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_json('offices.txt')

In [4]:
data = pd.DataFrame(df)

In [5]:
adresses_int = [int(re.findall('\d+',i)[0]) for i in data.address ]

In [6]:
new_data = data[['address','latitude','longitude']]

In [7]:
import datetime
from datetime import timedelta

In [8]:
min_lat = [55.419247, 56.184479]
max_lon = [36.853474, 38.377728]

**Создаем функцию для расчета географического расстояния**

In [9]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Радиус Земли в километрах
    R = 6371.0

    # Преобразование широты и долготы из градусов в радианы
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Разница между широтами и долготами
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Формула гаверсинуса для вычисления расстояния
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Расстояние между двумя точками
    distance = R * c

    return distance


**Функция для генерации исторических данных**

In [10]:
n = 1 #кол-во историй n-го поситителя
def rand_func1(n=1):
  lst = []
  for i in range(n):
    rand_point = [rd.randint(min_lat[0]*10**6, min_lat[1]*10**6)/10**6, rd.randint(max_lon[0]*10**6, max_lon[1]*10**6)/10**6]
    dis = [haversine(rand_point[0], rand_point[1], lat, lon) for lat, lon in data[['latitude','longitude']].values]
    data_1 = pd.DataFrame(data[['address','latitude','longitude']]).head(10)
    data_1['distance'] = dis[:10]
    data_1['user_lat'] = rand_point[0]
    data_1['user_lon'] = rand_point[1]
    rand = rd.randint(14,17)
    rand_visit = [rand] + [rand-3*(i+2) for i in range(3) ] + [rd.randint(0,5) for i in range(6)]

    id = [i+1 for j in range(10)]
    data_1['id'] = id
    data_1 = data_1[['id','distance','address','latitude','longitude','user_lat','user_lon']].sort_values(by=['distance']).head(10)
    data_1['visit_count'] = rand_visit
    data_1['date'] = pd.date_range(end='13/10/2023', periods=10,freq='W')#pd.date_range(start='10/4/2023', end='13/10/2023')
    moving_method = [rd.choice(['Пешком','Транспорт']) for i in range(10)]
    data_1['moving_method'] = moving_method

    office_loading = [rd.randint(0,1) for i in range(10)]
    data_1['office_loading'] = office_loading

    route_loading = [rd.randint(0,1) for i in range(10)]
    data_1['route_loading'] = route_loading

    parking_place = [rd.choice([True, False]) for i in range(10)]
    data_1['parking_place'] = parking_place

    #df_total= pd.concat([data_1])
    lst.append(data_1)
  return pd.concat([i for i in lst]) #(data_1[['id','dis','address','lat','lon','visit_count']].sort_values(by=['dis']).head(10))
df_total = rand_func1(17)

C:\Users\user\AppData\Local\Temp\ipykernel_18572\3984110777.py:34: UserWarning: Parsing '13/10/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_total = rand_func1(17)
C:\Users\user\AppData\Local\Temp\ipykernel_18572\3984110777.py:34: UserWarning: Parsing '13/10/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_total = rand_func1(17)
C:\Users\user\AppData\Local\Temp\ipykernel_18572\3984110777.py:34: UserWarning: Parsing '13/10/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_total = rand_func1(17)
C:\Users\user\AppData\Local\Temp\ipykernel_18572\3984110777.py:34: UserWarning: Parsing '13/10/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_total = rand_func1(17)
C:\Users\user\AppData\Local\Temp\ipykernel_18572\3984110777.py:34: UserWarning: Parsing 

In [56]:
label_encoders = {}
categorical_columns = ['moving_method']

for col in categorical_columns:
    le = LabelEncoder()
    df_total[col] = le.fit_transform(df_total[col])
    label_encoders[col] = le

In [60]:
df_total

,id,distance,address,latitude,longitude,user_lat,user_lon,visit_count,date,moving_method,office_loading,route_loading,parking_place,AverageDistanceToBranches
4,1,11.465022,"141207, Московская область, г. Пушкино, ул. Ту...",56.010849,37.854359,56.112408,37.822473,15,2023-08-06,0,0,1,False,0.053195
5,1,11.711543,"141200, Московская область, г. Пушкино, ул. Че...",56.008335,37.851467,56.112408,37.822473,9,2023-08-13,1,1,0,True,0.048416
8,1,16.389283,"141280, Московская область, г. Ивантеевка, ул....",55.973255,37.909462,56.112408,37.822473,6,2023-08-20,1,1,1,False,0.037745
9,1,22.688705,"141196, Московская область, г. Фрязино, пр-т М...",55.956615,38.058323,56.112408,37.822473,3,2023-08-27,1,1,0,True,0.028094
2,1,23.880550,"141730, Московская область, г. Лобня, ул. Лени...",56.012386,37.482059,56.112408,37.822473,0,2023-09-03,0,0,1,True,0.027721
3,1,36.249861,"142432, Московская область, г. Черноголовка, п...",56.011509,38.377728,56.112408,37.822473,3,2023-09-10,0,1,0,False,0.020155
6,1,40.087328,"124498, г. Москва, г. Зеленоград, корп. 401",56.000401,37.208775,56.112408,37.822473,2,2023-09-17,0,0,0,False,0.021088
7,1,43.551083,"124365, г. Москва, Зеленоград, ул. Каменка, ко...",55.979717,37.162693,56.112408,37.822473,4,2023-09-24,0,1,0,True,0.022218
0,1,52.530426,"141506, Московская область, г. Солнечногорск, ...",56.184479,36.984314,56.112408,37.822473,4,2023-10-01,1,0,1,False,0.029981
1,1,53.037974,"141500, Московская область, г. Солнечногорск, ...",56.183239,36.975700,56.112408,37.822473,1,2023-10-08,0,0,0,False,0.031306


**Функция для генерации 10 банков, которые были бы отобраны через жесткий фильтр**

In [51]:
n = 1 #кол-во историй n-го поситителя
def rand_func1(n=1):
  lst = []
  for i in range(n):
    rand_point = [rd.randint(min_lat[0]*10**6, min_lat[1]*10**6)/10**6, rd.randint(max_lon[0]*10**6, max_lon[1]*10**6)/10**6]
    dis = [haversine(rand_point[0], rand_point[1], lat, lon) for lat, lon in data[['latitude','longitude']].values]
    data_1 = pd.DataFrame(data[['address','latitude','longitude']]).head(10)
    data_1['distance'] = dis[:10]
    data_1['user_lat'] = rand_point[0]
    data_1['user_lon'] = rand_point[1]
    rand = rd.randint(14,17)
    rand_visit = [rand] + [rand-3*(i+2) for i in range(3) ] + [rd.randint(0,5) for i in range(6)]

    id = [i+1 for j in range(10)]
    data_1['id'] = id
    data_1 = data_1[['id','distance','address','latitude','longitude','user_lat','user_lon']].sort_values(by=['distance']).head(10)
    data_1['visit_count'] = rand_visit
    data_1['date'] = pd.date_range(end='13/10/2023', periods=10,freq='W')#pd.date_range(start='10/4/2023', end='13/10/2023')
    moving_method = [rd.choice(['Пешком','Транспорт']) for i in range(10)]
    data_1['moving_method'] = moving_method

    office_loading = [rd.randint(0,1) for i in range(10)]
    data_1['office_loading'] = office_loading

    route_loading = [rd.randint(0,1) for i in range(10)]
    data_1['route_loading'] = route_loading

    parking_place = [rd.choice([True, False]) for i in range(10)]
    data_1['parking_place'] = parking_place

    #df_total= pd.concat([data_1])
    lst.append(data_1)
  return pd.concat([i for i in lst]) #(data_1[['id','dis','address','lat','lon','visit_count']].sort_values(by=['dis']).head(10))
df_selected = rand_func1(1)

C:\Users\user\AppData\Local\Temp\ipykernel_18572\4256331920.py:34: UserWarning: Parsing '13/10/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df_selected = rand_func1(1)


In [54]:
# Преобразование категориальных переменных в числовые с помощью LabelEncoder
label_encoders = {}
categorical_columns = ['moving_method']

for col in categorical_columns:
    le = LabelEncoder()
    df_selected[col] = le.fit_transform(df_selected[col])
    label_encoders[col] = le

**Модель для корректировки 10 значений на основе исторических данных**

In [59]:
n_neighbors = 10
nn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
nn.fit(df_selected[['distance','latitude', 'longitude', 'user_lat',
       'user_lon', 'visit_count', 'office_loading',
       'route_loading', 'parking_place', 'moving_method']])

# Рассчитаем расстояния и индексы ближайших соседей для каждого посещения
distances, indices = nn.kneighbors(df_total[['distance','latitude', 'longitude', 'user_lat',
       'user_lon', 'visit_count', 'office_loading',
       'route_loading', 'parking_place', 'moving_method']])

# Найдем средние расстояния до ближайших отделений для каждого посещения
average_distances = np.mean(distances, axis=1)

# Добавим средние расстояния в исходный датафрейм
df_total['AverageDistanceToBranches'] = average_distances

# Отсортируем банки по средним расстояниям
df_selected['AverageDistance'] = np.mean(nn.kneighbors(df_selected[['distance','latitude', 'longitude', 'user_lat',
       'user_lon', 'visit_count', 'office_loading',
       'route_loading', 'parking_place','moving_method']])[0], axis=1)
sorted_selected_df = df_selected.sort_values(by='AverageDistance')

# Скорректируем список 10 отобранных банков на основе средних расстояний
recommended_branches = sorted_selected_df.head(10)

# Выведем скорректированный список 10 отобранных банков
recommended_branches

,id,distance,address,latitude,longitude,user_lat,user_lon,visit_count,date,moving_method,office_loading,route_loading,parking_place,AverageDistance
2,1,27.645601,"141730, Московская область, г. Лобня, ул. Лени...",56.012386,37.482059,56.167393,37.133627,2,2023-09-03,0,0,1,False,0.023366
4,1,47.979915,"141207, Московская область, г. Пушкино, ул. Ту...",56.010849,37.854359,56.167393,37.133627,5,2023-09-17,1,1,0,False,0.025316
5,1,47.916604,"141200, Московская область, г. Пушкино, ул. Че...",56.008335,37.851467,56.167393,37.133627,4,2023-09-10,0,1,0,True,0.025320
8,1,52.770159,"141280, Московская область, г. Ивантеевка, ул....",55.973255,37.909462,56.167393,37.133627,3,2023-09-24,1,1,0,True,0.030348
7,1,20.946436,"124365, г. Москва, Зеленоград, ул. Каменка, ко...",55.979717,37.162693,56.167393,37.133627,2,2023-08-27,1,0,0,True,0.030456
6,1,19.145081,"124498, г. Москва, г. Зеленоград, корп. 401",56.000401,37.208775,56.167393,37.133627,5,2023-08-20,0,1,1,True,0.032699
9,1,62.004389,"141196, Московская область, г. Фрязино, пр-т М...",55.956615,38.058323,56.167393,37.133627,5,2023-10-01,0,0,0,False,0.042527
1,1,9.932745,"141500, Московская область, г. Солнечногорск, ...",56.183239,36.975700,56.167393,37.133627,8,2023-08-13,1,1,0,True,0.052348
0,1,9.435144,"141506, Московская область, г. Солнечногорск, ...",56.184479,36.984314,56.167393,37.133627,14,2023-08-06,0,0,1,True,0.057264
3,1,79.099654,"142432, Московская область, г. Черноголовка, п...",56.011509,38.377728,56.167393,37.133627,3,2023-10-08,0,1,0,True,0.073183
